In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
#from src.data.sets import split_sets_random, save_sets
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from mlflow.models.signature import infer_signature

In [28]:
from platform import python_version
print(python_version())

3.7.6


In [4]:
df = pd.read_csv('../data/interim/Mall_Customers.csv')
df_cleaned = df.copy()

In [7]:
ohe = OneHotEncoder(sparse=False, drop='first')
df_cleaned['Gender'] = ohe.fit_transform(df_cleaned[['Gender']])

In [11]:
mlflow.set_tracking_uri('http://mlflow:5000')
mlflow.set_experiment('xgboost_spending')
run = mlflow.start_run()

INFO: 'xgboost_spending' does not exist. Creating a new experiment


In [14]:
mlflow.set_tag("model.description", "RandomForest with default hyperparameter")
mlflow.set_tag("model.version", "0.1")
mlflow.sklearn.autolog()

In [23]:
rf1 = RandomForestRegressor(random_state=8)
X_train = df_cleaned.iloc[:,:3]
y_train = df_cleaned.iloc[:, -1]
rf1.fit(X_train, y_train)

2022/03/09 11:06:27 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID '0baa4985e4c94478b70802732b2875df'
2022/03/09 11:06:28 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID '0baa4985e4c94478b70802732b2875df'
2022/03/09 11:06:28 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID '0baa4985e4c94478b70802732b2875df'
2022/03/09 11:06:28 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=8, verbose=0, warm_start=False)

In [25]:
signature = infer_signature(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/mlflow/models/signature.py:124: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  outputs = _infer_schema(model_output) if model_output is not None else None


In [26]:
mlflow.sklearn.log_model(rf1, artifact_path="model", 
                         signature=signature, 
                         registered_model_name="sklearn-rf-spending") 

Successfully registered model 'sklearn-rf-spending'.
2022/03/09 11:52:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sklearn-rf-spending, version 1
Created version '1' of model 'sklearn-rf-spending'.


In [27]:
mlflow.end_run()